In [2]:
import nba_py
import nba_py.game
import nba_py.player
import nba_py.team

import pandas as pd
import numpy as np
%matplotlib inline

import datetime
import time
import functools

import pymysql
from sqlalchemy import create_engine

import sys
sys.path.append('/Users/wonderui/OneDrive/6_Module_Package')
sys.path.append('/Users/WangRui/OneDrive/6_Module_Package')
import hoop_pwd
pwd = hoop_pwd.password

old_settings = np.seterr(all='print')
np.geterr()

{'divide': 'print', 'invalid': 'print', 'over': 'print', 'under': 'print'}

In [8]:
conn = create_engine('mysql+pymysql://root:%s@118.190.202.87:3306/nba_stats' % pwd)
game_stats_logs = pd.read_sql_table('game_stats_logs', conn)
game_stats_logs = game_stats_logs[game_stats_logs['GAME_TYPE'] != 'all_star']
print(str(len(game_stats_logs)) + ' player stats loaded.')

182297 player stats loaded.


In [9]:
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 1)
someday = datetime.date(2017, 1, 1)

all_players = nba_py.player.PlayerList(season = '2017-18').info()

In [10]:
def get_games(date):
    '''
    return a pandas.Series of all the games available on this day.
    '''
    return nba_py.Scoreboard(month = date.month, 
                             day = date.day, 
                             year = date.year, 
                             league_id = '00', 
                             offset = 0).game_header()[['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID']]

games = get_games(someday)
games

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID
0,0021600508,1610612737,1610612759
1,0021600509,1610612748,1610612765
2,0021600510,1610612754,1610612753
3,0021600511,1610612750,1610612757
4,0021600512,1610612747,1610612761


In [83]:
def get_players(games):
    home_team_player = all_players[all_players['TEAM_ID'].isin(games['HOME_TEAM_ID'])][['PERSON_ID', 'TEAM_ID']]
    home_team_player['Location'] = 'HOME'
    away_team_player = all_players[all_players['TEAM_ID'].isin(games['VISITOR_TEAM_ID'])][['PERSON_ID', 'TEAM_ID']]
    away_team_player['Location'] = 'AWAY'
    players = pd.concat([home_team_player, away_team_player])
    game_team = pd.concat([games[['HOME_TEAM_ID', 'GAME_ID']].rename(columns = {'HOME_TEAM_ID': 'TEAM_ID'}), 
                           games[['VISITOR_TEAM_ID', 'GAME_ID']].rename(columns = {'VISITOR_TEAM_ID': 'TEAM_ID'})])
    players = pd.merge(players, game_team, on = 'TEAM_ID')
    team_team = pd.concat([games[['HOME_TEAM_ID', 'VISITOR_TEAM_ID']].\
                           rename(columns = {'HOME_TEAM_ID': 'TEAM_ID', 'VISITOR_TEAM_ID': 'Against_Team_ID'}), 
                           games[['VISITOR_TEAM_ID', 'HOME_TEAM_ID']].\
                           rename(columns = {'VISITOR_TEAM_ID': 'TEAM_ID', 'HOME_TEAM_ID': 'Against_Team_ID'})])
    return pd.merge(players, team_team, on = 'TEAM_ID')
    
    

players = get_players(games)
players

,PERSON_ID,TEAM_ID,Location,GAME_ID,Against_Team_ID
0,1628389,1610612748,HOME,0021600509,1610612765
1,201609,1610612748,HOME,0021600509,1610612765
2,203580,1610612748,HOME,0021600509,1610612765
3,201961,1610612748,HOME,0021600509,1610612765
4,1627773,1610612748,HOME,0021600509,1610612765
5,2617,1610612748,HOME,0021600509,1610612765
6,201949,1610612748,HOME,0021600509,1610612765
7,204020,1610612748,HOME,0021600509,1610612765
8,1628571,1610612748,HOME,0021600509,1610612765
9,203585,1610612748,HOME,0021600509,1610612765


In [62]:
def get_last_n_game_logs(player_ID, game_ID, n):
    '''
    retrun a pandas.DataFrame of a player's last n game logs before a specific game.
    '''
    player_game_logs = game_stats_logs[game_stats_logs['PLAYER_ID'] == player_ID]
    last_n_game = player_game_logs[player_game_logs['GAME_ID_O'] < game_ID].sort_values('GAME_ID_O').tail(n)
    return last_n_game[['MINS', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 'TO', 'FGM', 'FGA', 'FG3M']]

test_p = get_last_n_game_logs(201609, '1600200509', 5)
test_p

,MINS,PTS,AST,OREB,DREB,STL,BLK,TO,FGM,FGA,FG3M
160088,35.90,21.0,7.0,0.0,2.0,0.0,0.0,3.0,6.0,15.0,3.0
160334,33.52,23.0,5.0,0.0,4.0,0.0,0.0,4.0,10.0,16.0,2.0
160947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161289,32.08,5.0,8.0,0.0,3.0,0.0,0.0,4.0,2.0,11.0,0.0
161499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
def get_score_36(game_logs):
    '''
    return the Fantasy [0]score and [1]cov of the given game logs(pandas.DataFrame).
    '''
    convert_to_36 = lambda x: x[['PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 
                                 'TO', 'FGM', 'FGA', 'FG3M']] * 36 / x['MINS']
    stats = game_logs.apply(convert_to_36, axis = 1)#.mean()
    stats['SCO'] = stats['PTS'] * 1 + stats['AST'] * 1.5 + \
    stats['OREB'] * 1 + stats['DREB'] * 0.7 + \
    stats['STL'] * 2 + stats['BLK'] * 1.8 + stats['TO'] * -1 + \
    stats['FGM'] * 0.4 + (stats['FGA'] - stats['FGM']) * -1 + stats['FG3M'] * 0.5
    return stats['SCO'].mean(), stats['SCO'].std()/stats['SCO'].mean()

get_score_36(test_p)

(21.002005891328675, 0.56233251512117999)

In [76]:
def get_ma(row, n):
    '''
    return the average score of last n games.
    '''
    player_ID = row['PERSON_ID']
    game_ID_O = row['GAME_ID'][3:5] + row['GAME_ID'][:3] + row['GAME_ID'][-5:]
    ma_n = get_score_36(get_last_n_game_logs(player_ID, game_ID_O, n))[0]
    return round(float(ma_n), 2)

get_ma(players[players['PERSON_ID'] == 201609].iloc[0], 5)

21.0

In [77]:
def get_min(row, n):
    '''
    return the average mins played of last n games played.
    '''
    player_ID = row['PERSON_ID']
    game_ID_O = row['GAME_ID'][3:5] + row['GAME_ID'][:3] + row['GAME_ID'][-5:]
    min_n = get_last_n_game_logs(player_ID, game_ID_O, n)['MINS'].mean()
    return round(float(min_n), 2)

get_min(players[players['PERSON_ID'] == 201609].iloc[0], 5)

33.83

In [78]:
def get_min_cov(row, n):
    '''
    return coefficient of variance of mins
    '''
    player_ID = row['PERSON_ID']
    game_ID_O = row['GAME_ID'][3:5] + row['GAME_ID'][:3] + row['GAME_ID'][-5:]
    min_cov_n = get_last_n_game_logs(player_ID, game_ID_O, n)['MINS'].std()/\
    get_last_n_game_logs(player_ID, game_ID_O, n)['MINS'].mean()
    return round(float(min_cov_n), 3)

get_min_cov(players[players['PERSON_ID'] == 201609].iloc[0], 5)

0.057

In [81]:
def get_sco_cov(row, n):
    '''
    return coefficient of variance of sco
    '''
    player_ID = row['PERSON_ID']
    game_ID_O = row['GAME_ID'][3:5] + row['GAME_ID'][:3] + row['GAME_ID'][-5:]
    sco_cov_n = get_score_36(get_last_n_game_logs(player_ID, game_ID_O, n))[1]
    return round(float(sco_cov_n), 3)

get_sco_cov(players[players['PERSON_ID'] == 201609].iloc[0], 5)

0.562

In [96]:
players['MA_20'] = players.apply(lambda x: get_ma(x, 20), axis = 1)
players['MA_10'] = players.apply(lambda x: get_ma(x, 10), axis = 1)
players['MA_5'] = players.apply(lambda x: get_ma(x, 5), axis = 1)
players['MIN_20'] = players.apply(lambda x: get_min(x, 20), axis = 1)
players['MIN_10'] = players.apply(lambda x: get_min(x, 10), axis = 1)
players['MIN_5'] = players.apply(lambda x: get_min(x, 5), axis = 1)
players['MIN_COV_20'] = players.apply(lambda x: get_min_cov(x, 20), axis = 1)
players['SCO_COV_20'] = players.apply(lambda x: get_sco_cov(x, 20), axis = 1)

players['EXP_SCO'] = round(players[['MA_20', 'MA_10', 'MA_5']].mean(axis = 1) * \
                           players[['MIN_20', 'MIN_10', 'MIN_5']].mean(axis = 1) / 36, 2)
players

,PERSON_ID,TEAM_ID,Location,GAME_ID,Against_Team_ID,MA_20,MA_10,MA_5,MIN_20,MIN_10,MIN_5,MIN_COV_20,SCO_COV_20,EXP_SCO
0,1628389,1610612748,HOME,0021600509,1610612765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201609,1610612748,HOME,0021600509,1610612765,28.52,26.21,21.00,36.01,36.06,33.83,0.094,0.420,24.75
2,203580,1610612748,HOME,0021600509,1610612765,11.18,12.62,15.20,17.32,18.17,15.11,0.401,1.361,6.09
3,201961,1610612748,HOME,0021600509,1610612765,17.84,16.47,14.10,27.38,22.96,20.37,0.317,0.526,10.57
4,1627773,1610612748,HOME,0021600509,1610612765,16.71,16.86,6.80,5.17,5.09,3.29,0.841,0.910,1.69
5,2617,1610612748,HOME,0021600509,1610612765,17.85,0.00,0.00,4.49,4.58,4.58,0.768,0.864,0.75
6,201949,1610612748,HOME,0021600509,1610612765,27.06,31.01,34.61,24.82,23.83,22.96,0.201,0.575,20.48
7,204020,1610612748,HOME,0021600509,1610612765,24.79,26.67,27.45,32.05,31.22,30.18,0.101,0.413,22.76
8,1628571,1610612748,HOME,0021600509,1610612765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,203585,1610612748,HOME,0021600509,1610612765,13.28,12.08,13.67,30.68,27.21,26.18,0.228,0.460,10.13


In [115]:
players[players['EXP_SCO'].notnull()].sort_values('EXP_SCO')

,PERSON_ID,TEAM_ID,Location,GAME_ID,Against_Team_ID,MA_20,MA_10,MA_5,MIN_20,MIN_10,MIN_5,MIN_COV_20,SCO_COV_20,EXP_SCO
166,1627740,1610612765,AWAY,0021600509,1610612748,-2.90,-0.32,-23.16,4.30,4.62,3.35,0.460,-9.877,-1.00
135,1627774,1610612757,AWAY,0021600511,1610612750,-0.56,-6.68,-7.83,5.01,4.46,2.61,0.757,-27.233,-0.56
160,1627832,1610612761,AWAY,0021600512,1610612747,23.01,1.31,-31.58,5.60,8.87,2.28,1.366,1.723,-0.38
116,1627854,1610612759,AWAY,0021600508,1610612737,5.84,-4.35,-10.65,6.44,2.66,1.54,1.162,3.367,-0.30
91,1627826,1610612747,HOME,0021600512,1610612761,3.26,-8.93,2.06,8.13,7.33,9.82,0.784,5.010,-0.28
51,1626202,1610612754,HOME,0021600510,1610612753,9.15,-11.80,NaN,5.75,3.05,NaN,0.593,3.347,-0.16
154,1626259,1610612761,AWAY,0021600512,1610612747,0.00,0.00,0.00,0.20,0.20,0.20,NaN,NaN,0.00
162,1626153,1610612761,AWAY,0021600512,1610612747,14.81,-13.69,NaN,6.38,2.63,NaN,0.669,1.379,0.07
150,203909,1610612761,AWAY,0021600512,1610612747,10.27,-1.19,6.92,4.12,4.22,4.59,0.571,3.002,0.64
5,2617,1610612748,HOME,0021600509,1610612765,17.85,0.00,0.00,4.49,4.58,4.58,0.768,0.864,0.75
